In [0]:
%pip install great_expectations==1.10.0 typing_extensions==4.8.0 --upgrade

In [0]:
dbutils.library.restartPython()

In [0]:
import os
import sys

# 重新计算并注入路径
current_dir = os.getcwd()
# 关键改动：先跳出 tests (..)，再进入 src/sbit_medallion_pipeline
src_path = os.path.abspath(os.path.join(current_dir, "..", "src", "sbit_medallion_pipeline"))

if src_path not in sys.path:
    sys.path.insert(0, src_path)
    print(f"✅ 路径已注入: {src_path}")

# 验证文件是否存在（调试用）
if os.path.exists(os.path.join(src_path, "great_expectations_common.py")):
    print("✅ 找到 great_expectations_common.py 文件")
else:
    print(f"❌ 未找到文件，请检查路径: {src_path}")

In [ ]:
# 接收来自 Databricks Bundle 的参数
dbutils.widgets.text("catalog", "sbit_test_catalog")
dbutils.widgets.text("schema", "sbit_db_integration_test")
dbutils.widgets.text("env", "test") # 新增环境标识

test_catalog = dbutils.widgets.get("catalog")
test_db = dbutils.widgets.get("schema")
current_env = dbutils.widgets.get("env")

print(f"🚀 运行环境：{test_catalog}.{test_db} (Mode: {current_env})")

In [0]:
# --- 2. 初始化测试 Catalog 和 Schema (占位) ---

spark.sql(f"CREATE CATALOG IF NOT EXISTS {test_catalog}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {test_catalog}.sbit_db") 
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {test_catalog}.{test_db}") # 实际运行库

In [0]:
dbutils.widgets.text("env", "test", "Environment")
env = dbutils.widgets.get("env")

In [0]:
# --- 2. 加载配置与类定义 ---

In [0]:
%run "../src/sbit_medallion_pipeline/01_config"

In [0]:
%run "../src/sbit_medallion_pipeline/02_setup"

In [0]:
%run "../src/sbit_medallion_pipeline/03_history_loader"

In [0]:
%run "../src/sbit_medallion_pipeline/04_bronze"

In [0]:
%run "../src/sbit_medallion_pipeline/05_silver"

In [0]:
%run "../src/sbit_medallion_pipeline/06_gold"

In [0]:
import great_expectations_common as gx_common

In [0]:
class TestConfig(Config):
    def __init__(self):
        super().__init__()
        self.test_root = self.project_dir + "test_automation_temp/"
        self.landing = self.test_root + "landing/"
        self.checkpoint = self.test_root + "checkpoints/"
        self.medallion = self.test_root + "medallion/"
        self.db_name = test_db
        self.catalog = test_catalog

In [0]:
test_conf = TestConfig()
env = "test"

In [0]:
# --- 6. 深度全量补丁函数 ---
def apply_test_patch(obj, t_conf):
    """
    强制重写对象中所有关联生产路径的属性，实现 100% 隔离。
    """
    obj.catalog = t_conf.catalog
    obj.db_name = t_conf.db_name
    
    # 定义需要覆盖的所有潜在属性名
    paths_to_patch = {
        "medallion": t_conf.medallion,
        "checkpoint_base": t_conf.checkpoint,
        "landing_zone": t_conf.landing + "raw",
        "initial": t_conf.medallion + "initial",
        "bronze": t_conf.medallion + "bronze",
        "silver": t_conf.medallion + "silver",
        "gold": t_conf.medallion + "gold",
        "config_gx": t_conf.gx_configs
    }
    
    for attr, value in paths_to_patch.items():
        if hasattr(obj, attr):
            setattr(obj, attr, value)
      
    return obj

In [0]:
# --- 7. 实例化对象并打补丁 ---
# 此时 Catalog 已创建，__init__ 里的 USE 语句将不再报错
print("🚀 正在实例化组件并注入补丁...")

setup = apply_test_patch(SetupHelper(env), test_conf)
hl = apply_test_patch(HistoryLoader(env), test_conf)
bronze = apply_test_patch(Bronze(env), test_conf)
silver = apply_test_patch(Silver(env), test_conf)
gold = apply_test_patch(Gold(env), test_conf)

# 校验路径安全性
if "test_automation_temp" not in setup.medallion:
    raise Exception("🚨 安全检查失败！路径未正确重定向至隔离目录，请检查 apply_test_patch 函数。")

In [0]:
# --- 5. 【核心安全检查：多维度路径熔断】 ---

def safety_check(obj, attr_name):
    """
    检查对象的特定属性是否包含隔离关键字。
    如果属性指向了非测试目录，直接抛出异常，防止误删生产数据。
    """
    if hasattr(obj, attr_name):
        value = getattr(obj, attr_name)
        
        # 如果是路径属性
        if any(keyword in attr_name.lower() for keyword in ["medallion", "checkpoint", "landing", "path", "initial", "bronze", "silver", "gold"]):
            if "test_automation_temp" not in str(value):
                raise Exception(f"🚨 安全熔断！对象 {obj.__class__.__name__} 的路径属性 '{attr_name}' 危险: {value} (缺少 test_automation_temp 关键字)")
        
        # 如果是数据库属性
        elif "db_name" in attr_name:
            if "integration_test" not in str(value):
                raise Exception(f"🚨 安全熔断！对象 {obj.__class__.__name__} 的数据库属性 '{attr_name}' 危险: {value} (缺少 integration_test 关键字)")
        
        # 如果是 Catalog 属性
        elif "catalog" in attr_name:
            if "test_catalog" not in str(value) and "dev_catalog" not in str(value):
                 # 这里允许 dev_catalog，因为我们是通过 DB Name 隔离的，但可以根据需要收紧
                 pass

        print(f"✅ 安全校验通过: {obj.__class__.__name__}.{attr_name} -> {value}")

print("🚀 开始深度路径隔离审计...")

# 定义所有需要被审计的属性列表
critical_attrs = [
    "db_name", 
    "catalog", 
    "medallion", 
    "checkpoint_base", 
    "landing_zone", 
    "initial", 
    "bronze", 
    "silver", 
    "gold"
]

# 执行审计
for o in [setup, hl, bronze, silver, gold]:
    for attr in critical_attrs:
        safety_check(o, attr)

print("🌟 审计完成！所有组件均已安全锁定在隔离区。")

In [0]:
print(f"⚠️ 隔离目录: {test_conf.test_root}")
print(f"⚠️ 测试数据库: {test_conf.catalog}.{test_conf.db_name}")

In [0]:
# --- 2. 基础设施初始化 (执行 Setup) ---
print("🛠️ 正在初始化测试基础设施 (setup.setup)...")
setup.cleanup() # 确保环境干净
dbutils.fs.rm(test_conf.landing, True)
dbutils.fs.rm(f'{test_conf.test_root}gx', True)

In [0]:
setup.setup()   # 创建数据库、目录、Volumes
setup.validate() # 验证目录是否创建成功

In [0]:
# --- 修改后的测试环境准备脚本 ---

# A. 确保 gx 库（Schema）存在
gx_schema = f"{setup.catalog}.gx"
print(f"🛠️ 确保测试环境 GX 库存在: {gx_schema}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gx_schema}")

# B. 定义并创建检疫表
# 建议与 gec 逻辑保持一致，使用 {catalog}.gx.data_quality_quarantine
quarantine_table = f"{gx_schema}.data_quality_quarantine"

print(f"🛠️ 正在创建测试专用检疫表: {quarantine_table}")

create_table_sql = f"""
CREATE TABLE IF NOT EXISTS {quarantine_table} (
    table_name STRING,
    gx_batch_id STRING,
    violated_rules STRING,
    raw_data STRING,
    ingestion_time TIMESTAMP
)
USING DELTA
LOCATION '{test_conf.test_root}gx/quarantine/'
"""
spark.sql(create_table_sql)

# 验证创建结果
table_exists = spark.sql(f"SHOW TABLES IN {gx_schema} LIKE 'data_quality_quarantine'").count() > 0

if table_exists:
    print(f"✅ 检疫表已就绪: {quarantine_table}")
else:
    raise Exception(f"❌ 检疫表创建失败: {quarantine_table}")

In [0]:
# --- 4. 模拟 5 个数据源 (直接替换版：强行推进水印) ---
import json

# 确保路径结尾斜杠处理正确
bz_landing = test_conf.landing.rstrip('/') + "/raw/"

# A. Registered Users (保持不变)
reg_users = """user_id,device_id,mac_address,registration_timestamp
user1_070be4a0,device1_35c064c9,84:20:e7:59:a0:07,1705445488.515213
user1_51141330,device1_d09bea2a,76:e4:1b:f6:8f:cd,1717627888.515229
user1_661d51db,device1_65bde85f,2c:a1:09:a1:c8:f5,1705272688.515239
,,BAD_MAC,INVALID_TIMESTAMP_FORMAT
"""
dbutils.fs.put(f"{bz_landing}registered_users_bz/1-registered_users_test.csv", reg_users, True)

# B. Gym Logins (保持不变)
gym_logins = """mac_address,gym,login,logout
76:e4:1b:f6:8f:cd,105,1767433888,1767440908
36:ac:71:c9:74:d1,102,1767466288,1767470848
cc:52:18:1d:bf:f2,109,1767430288,1767436588
"""
dbutils.fs.put(f"{bz_landing}gym_logins_bz/5-gym_logins_test.csv", gym_logins, True)

# C, D, E. Kafka Multiplex (JSON)
# 核心改动：增加足够晚的数据推进水印
kafka_events = [
    # 1. User Info (用于关联 user_bins)
    {
        "key": "user1_51141330", "topic": "user_info", "partition": 0, "offset": 202, "timestamp": "2026-01-04T07:33:55.211Z",
        "value": json.dumps({"user_id": "user1_51141330", "dob": "2003-06-18", "sex": "M", "gender": "Non-binary", "first_name": "Caitlin", "last_name": "Howard", "address": {"street_address": "0492 Maria Ridges Suite 776", "city": "Boltonside", "state": "MO", "zip": 46807}, "timestamp": "2026-01-04 06:51:28", "update_type": "new"})
    },
    # 2. BPM 1 & 2: 落在运动时间 17:56 - 19:43 之内 (用于生成聚合结果)
    {
        "key": "device1_d09bea2a", "topic": "bpm", "partition": 0, "offset": 670, "timestamp": "2026-01-04T07:33:53.681Z",
        "value": json.dumps({"device_id": "device1_d09bea2a", "time": "2026-01-03 18:30:00", "heartrate": 151.93})
    },
    {
        "key": "device1_d09bea2a", "topic": "bpm", "partition": 0, "offset": 671, "timestamp": "2026-01-04T07:33:53.681Z",
        "value": json.dumps({"device_id": "device1_d09bea2a", "time": "2026-01-03 19:00:00", "heartrate": 145.50})
    },
    # 3. 【核心补丁】BPM 3: 落在运动时间之后 (21:00:00)
    # 这条数据不仅要晚，而且必须让 Silver 层也处理它，从而产生一个晚的 end_time 传给 Gold
    {
        "key": "device1_d09bea2a", "topic": "bpm", "partition": 0, "offset": 672, "timestamp": "2026-01-04T07:33:53.681Z",
        "value": json.dumps({"device_id": "device1_d09bea2a", "time": "2026-01-03 21:00:00", "heartrate": 80.0})
    },
    # 4. Workout 记录 (17:56 - 19:43)
    {
        "key": "workout_id1_e5638c25", "topic": "workout", "partition": 0, "offset": 346, "timestamp": "2026-01-04T07:33:57.552Z",
        "value": json.dumps({"user_id": "user1_51141330", "workout_id": "workout_id1_e5638c25", "time": "2026-01-03 17:56:28", "action": "start", "session_id": "session1_239a2270"})
    },
    {
        "key": "workout_id1_e5638c25", "topic": "workout", "partition": 0, "offset": 347, "timestamp": "2026-01-04T07:33:57.552Z",
        "value": json.dumps({"user_id": "user1_51141330", "workout_id": "workout_id1_e5638c25", "time": "2026-01-03 19:43:28", "action": "stop", "session_id": "session1_239a2270"})
    },
    # 5. 【辅助踢腿】再增加一个新的运动，时间戳设为 21:00，确保彻底关掉之前的窗口
    {
        "key": "workout_id_kicker", "topic": "workout", "partition": 0, "offset": 348, "timestamp": "2026-01-04T07:33:57.552Z",
        "value": json.dumps({"user_id": "user1_51141330", "workout_id": "kicker", "time": "2026-01-03 21:00:00", "action": "start", "session_id": "s_kick"})
    }
]

dbutils.fs.put(f"{bz_landing}kafka_multiplex_bz/kafka_test_data.json", json.dumps(kafka_events), True)

print(f"✅ 增强版数据已准备就绪。")
print("🚀 包含 21:00 的运动和心率数据，将强制触发 19:43 窗口的输出。")

In [0]:
# --- 9. 全链路运行与校验 ---
print("🏃 运行 Pipeline...")
hl.load_history()

In [0]:
once = True
processing_time = "5 seconds"

In [0]:
# 验证 Bronze 类实际读取的路径
print(f"Bronze 预期的注册用户路径: {bronze.landing_zone}/registered_users_bz")

# 检查该路径下是否有文件
try:
    files = dbutils.fs.ls(f"{bronze.landing_zone}/registered_users_bz")
    for f in files:
        print(f"找到文件: {f.path} (大小: {f.size} bytes)")
except Exception as e:
    print(f"❌ 路径不存在或无法访问: {e}")

In [0]:
bronze.consume(once, processing_time)


In [0]:
silver.upsert()


In [0]:
gold.upsert()

In [0]:
# --- 9. 结果验证 (增加检疫表断言) ---
print("🧐 结果验证...")

# A. 验证 Gold 层汇总数据
summary_table = f"{setup.catalog}.{setup.db_name}.workout_bpm_summary"
res_count = spark.table(summary_table).count()

# B. 验证检疫表 (Quarantine) 数据
# 根据之前的模拟数据，我们在 Registered Users 中放入了空值和坏时间戳，应该被 GX 拦截
quarantine_table = f"{gx_schema}.data_quality_quarantine"
q_count = spark.table(quarantine_table).count()

print(f"📊 Gold 层记录数: {res_count}")
print(f"🛡️ 检疫表记录数: {q_count}")

# 执行断言逻辑
if res_count > 0:
    print(f"✨ 业务链路校验通过：在 {summary_table} 中成功产出数据。")
else:
    raise Exception("❌ 测试失败：Gold 层未发现数据结果。")

if q_count > 0:
    print(f"✅ 质量控制校验通过：坏数据已被正确拦截至 {quarantine_table}。")
    spark.table(quarantine_table).select("table_name", "violated_rules").show()
else:
    # 如果你预期有坏数据但检疫表为空，说明 GX 可能没起作用
    raise Exception("❌ 测试失败：预期的坏数据未进入检疫表，请检查 GX 配置。")

print("⭐ 集成测试所有断言均已通过！")

In [0]:
# 只有在环境为 test 且 Catalog 匹配时才执行删除
if current_env == "test" and test_catalog == "sbit_test_catalog":
    print(f"🔥 正在全量清空测试环境: {test_catalog}")

    # 1. 强制清理 Catalog
    try:
        spark.sql(f"DROP CATALOG IF EXISTS {test_catalog} CASCADE")
        print(f"✅ Catalog {test_catalog} 已销毁")
    except Exception as e:
        print(f"⚠️ Catalog 删除失败: {e}")

    # 2. 调用 setup 逻辑 (如果 setup 对象已实例化)
    try:
        setup.cleanup()
    except:
        pass

    # 3. 物理删除隔离路径 (确保路径包含安全关键字)
    if "test_automation_temp" in test_conf.test_root:
        print(f"🧹 正在删除物理路径: {test_conf.test_root}")
        dbutils.fs.rm(test_conf.test_root, True)

    print("✨ 测试沙箱已完全抹除。")
else:
    print(f"✅ 非集成测试环境 ({current_env})，保留数据供手动检查。")